In [ ]:
# --- INSTALLATION ---

!pip install pandas numpy matplotlib nltk scikit-learn transformers datasets torch
!kaggle datasets download -d shanegerami/ai-vs-human-text
!unzip -n ai-vs-human-text.zip
!rm ai-vs-human-text.zip

# -------------------------

In [ ]:
# --- IMPORTS ---

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stopwords = set(stopwords.words('english'))
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding
from datasets import Dataset
import torch

# -------------------------

In [41]:
# --- USEFUL FUNCTIONS ----

def clean_text(text):
    """
    This funtion get's rid of nonalphabetical characters, stopwords and lower cases the text.

    Args:
    text (str): The text to be cleaned

    Returns:
    text (str): The cleaned text

    Example:
    df['text'] = df['text'].apply(clean_text)
    """
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    text = text.lower()
    words = text.split()
    text = [word for word in words if not word in stopwords]
    text = ' '.join(words)
    return text

def tokenize_function(dataframe):
    """
    This funtion tokenizes the 'text' field of the dataframe.

    Args:
    dataframe (pandas.DataFrame): The dataframe to be tokenized

    Returns:
    dataframe (pandas.DataFrame): The tokenized dataframe

    Example and output:
    train_dataset_token = train_dataset.map(tokenize_function, batched=True)
    """
    return tokenizer(dataframe["text"], truncation=True)

def compute_metrics(eval_pred):
    """
    This funtion computes the accuracy, precision, recall and f1 score of the model.

    It'is passed to the trainer and it outputs when evaluating the model.

    Args:
    eval_pred (tuple): The predictions and labels of the model

    Returns:
    dict: The accuracy, precision, recall and f1 score of the model

    Example:
    >>> trainer.evaluate()
    {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }
    """
    predictions, labels = eval_pred
    predictions = predictions.argmax(axis=-1)
    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='binary')
    return {
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

# -------------------------

In [42]:
# --- LOADING THE MODEL ---

# Load the initial tokenizer and model to set the number of labels its going to classify as 2
checkpoint = "diegovelilla/EssAI"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint)

# -------------------------

In [ ]:
# --- DATA PREPROCESSING ---

df = pd.read_csv('AI_Human.csv')

# Separate human from ai
df_human = df[df["generated"] == 0]
df_ai = df[df["generated"] == 1]

# We take as many human written esssays as AI generate since the dataset is a bit unbalanced
df_ai_len = df_ai["text"].count()
df_human = df_human.sample(n=df_ai_len)

# We concatenate both dataframes, shuffle them and then we take 1% of them since those will be enough to fine tune the model
# and with my current resources I won't be able to process more. For better results increase the fraction of the data used.
df_unshuffled = pd.concat([df_human, df_ai])
df = df_unshuffled.sample(frac=0.01).reset_index(drop=True)

# Get rid of nonalphatetical characters, stopwords and we lower case it.
df['text'] = df['text'].apply(clean_text)

# According to the transformers library of hugging face the targets column name should be labels and ints
df = df.rename(columns={'generated': 'labels'})
df['labels'] = df['labels'].astype(int)

# We convert the pandas dataframe into a hugging face dataset and tokenize both of them
ds = Dataset.from_pandas(df)
ds_token = ds.map(tokenize_function, batched=True)

# Drop columns that are not necessary and set the dataset format to pytorch tensors
ds_token = ds_token.remove_columns(["text", "token_type_ids"])
ds_token.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# -------------------------


In [50]:
# --- INSTANTIATING TRAINER ----

# We instantiate a DataCollatorWithPadding in order to pad the inputs in batches while training
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

# Create the training arguments
training_args = TrainingArguments("./results")

# Create the trainer
trainer = Trainer(
    model,
    training_args,
    eval_dataset=ds_token,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics = compute_metrics
)

# -------------------------

In [ ]:
# --- EVALUATION ---

evaluation_results = trainer.evaluate()

print("Accuracy:", evaluation_results['eval_accuracy'])
print("Precision:", evaluation_results['eval_precision'])
print("Recall:", evaluation_results['eval_recall'])
print("F1:", evaluation_results['eval_f1'])

# -------------------------